In [111]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import plotly as plt
import numpy as np
np.random.seed(6615)

## Data

Based on this dataset, we have formed `coding3_dataH.csv`, which is structured as follows:

It contains 506 rows, corresponding to n=506

- There are 241 columns in total. The first column represents Y
- The subsequent 240 columns relate to the NCS basis functions for each of the 13 X variables. The number of knots are individually determined for each feature.

In [112]:
data = pd.read_csv("Coding3_dataH.csv", delimiter=",", header=0)
print(data.shape)

(505, 241)


## Task 1: Ridgeless Function

In [113]:
# accepts training and test datasets and returns the training and test errors of the ridgeless estimator. 
# For both datasets, the initial column represents the response vector Y
def ridgeless_regression(train, test):
    # Extract the response vector from the training and test datasets
    y_train = train.iloc[:,0]
    y_test = test.iloc[:,0]
    
    # Extract the feature matrix from the training and test datasets
    X_train = train.iloc[:,1:]
    X_test = test.iloc[:,1:]
    
    # center the design matrix
    X_train_center = X_train - X_train.mean()

    # use PCA to reduce the dimensionality of the feature matrix
    pca = PCA(n_components=None)
    X_train_pca = pca.fit_transform(X_train_center)

    # For computation stability, you need to exclude directions with extremely small eigenvalues (in PCA), eps = 1e-10
    keep = np.where(pca.explained_variance_ > 1e-10)[0]
    X_train_pca = X_train_pca[:,keep]

    # Calculate the beta coefficients and beta0 intercept
    beta = np.linalg.inv(X_train_pca.T.dot(X_train_pca)).dot(X_train_pca.T).dot(y_train)
    beta0 = y_train.mean()

    # calculate y_hat and error for train data
    y_hat_train = X_train_pca.dot(beta)
    train_error = np.sum((y_train - y_hat_train)**2) / len(y_train)

    # PCA transform the test data
    X_test_pca = pca.transform(X_test - np.mean(X_train))[:, keep]
    
    # calculate y_hat and error for test data
    y_hat_test = X_test_pca.dot(beta) + beta0
    test_error = np.sum((y_test - y_hat_test)**2) / len(y_test)
    
    return train_error, test_error

#split data into train and test
train, test = train_test_split(data, test_size=0.75)
ridgeless_regression(train, test)

(8.936072419454163, 0.11976737028598734)

## Task 2: Simulation study

Execute the procedure below for T=30
 times.

In each iteration,

- Randomly partition the data into training (25%) and test (75%).
- Calculate and log the test error from the ridgeless method using the first d columns of myData, where d ranges from 6 to 241. Keep in mind that the number of regression parameters spans from 5 to 240 because the first column represents Y

In [114]:
import plotly.express as px

T = 30
d_range = range(6, 241)
error_matrix = np.matrix(np.zeros((T, len(d_range))))

for t in range(T):
    # store the log_test_error in each iteration and plot it later vs. d
    test_error_arr = []
    for d in d_range:
    # split the data into training and test datasets, take the first d columns of data as the feature matrix
        train, test = train_test_split(data.iloc[:, :d], test_size=0.75)
        train_error, test_error = ridgeless_regression(train, test)
        error_matrix[t - 1, d - 6] = test_error

# find the median of each column of error_matrix, then take a log of it
log_median_test_error = np.array(np.log(np.median(error_matrix, axis=0))).flatten()

# plot the log_test_error vs. d
fig = px.scatter(x=d_range, y=log_median_test_error, labels={'x':'num of features d', 'y':'Log of Test Error'})
fig.show()